In [ ]:
create database agridata;
use agridata;

#1.Year-wise Trend of Rice Production Across States (Top 3)
SELECT year,`state name`,riceproduction
FROM ( SELECT year,`state name`,SUM(`rice production`) AS riceproduction,ROW_NUMBER() OVER (PARTITION BY year ORDER BY SUM(`rice production`) DESC, `state name`) AS rowno
    FROM agriinfo GROUP BY year, `state name`) AS f WHERE rowno <= 3;
	 
SELECT year,`state name`,`rice production`
FROM (SELECT year,`state name`,SUM(`rice production`) AS `rice production`,DENSE_RANK() OVER (PARTITION BY year ORDER BY SUM(`rice production`) DESC) AS rowno
FROM agriinfo GROUP BY year, `state name`) AS f WHERE rowno <= 3 ORDER BY year ASC, rowno ASC;

#2.Top 5 Districts by Wheat Yield increase Over the Last 5 Years
SELECT year,`dist name`, wheatyield
FROM ( SELECT year,`dist name`, AVG(`wheat yield`) AS wheatyield, ROW_NUMBER() OVER (PARTITION BY year ORDER BY AVG(`wheat yield`) DESC) AS rono
    FROM agriinfo WHERE year >= (SELECT MAX(year) - 5 FROM agriinfo) GROUP BY year, `dist name`
) AS j WHERE rono <= 5 ORDER BY year ASC;

#3.States with the Highest Growth in Oilseed Production (5-Year Growth Rate)
	SELECT
    a1.`state name`,
    SUM(a1.`oilseeds production`) AS latest_production,
    SUM(a2.`oilseeds production`) AS old_production,
    ((SUM(a1.`oilseeds production`) - SUM(a2.`oilseeds production`)) / SUM(a2.`oilseeds production`)) * 100 AS growth_rate
FROM agriinfo a1 JOIN agriinfo a2 ON a1.`state name` = a2.`state name`
WHERE a1.year = (SELECT MAX(year) FROM agriinfo) AND a2.year = (SELECT MAX(year) - 5 FROM agriinfo)
GROUP BY a1.`state name`
ORDER BY growth_rate DESC
LIMIT 5;

#4.District-wise Correlation Between Area and Production for Major Crops (Rice, Wheat, and Maize)
	SELECT
    `dist name`,
    (SUM(`rice area` * `rice production`) - (SUM(`rice area`) * SUM(`rice production`)) / COUNT(*)) /
    (SQRT(SUM(`rice area` * `rice area`) - (SUM(`rice area`) * SUM(`rice area`)) / COUNT(*)) *
     SQRT(SUM(`rice production` * `rice production`) - (SUM(`rice production`) * SUM(`rice production`)) / COUNT(*))) AS rice_correlation,

    (SUM(`wheat area` * `wheat production`) - (SUM(`wheat area`) * SUM(`wheat production`)) / COUNT(*)) /
    (SQRT(SUM(`wheat area` * `wheat area`) - (SUM(`wheat area`) * SUM(`wheat area`)) / COUNT(*)) *
     SQRT(SUM(`wheat production` * `wheat production`) - (SUM(`wheat production`) * SUM(`wheat production`)) / COUNT(*))) AS wheat_correlation,

    (SUM(`maize area` * `maize production`) - (SUM(`maize area`) * SUM(`maize production`)) / COUNT(*)) /
    (SQRT(SUM(`maize area` * `maize area`) - (SUM(`maize area`) * SUM(`maize area`)) / COUNT(*)) *
     SQRT(SUM(`maize production` * `maize production`) - (SUM(`maize production`) * SUM(`maize production`)) / COUNT(*))) AS maize_correlation
FROM agriinfo GROUP BY `dist name`
ORDER BY `dist name`
LIMIT 0, 1000;

#5.Yearly Production Growth of Cotton in Top 5 Cotton Producing States
WITH TopCottonStates AS (
    SELECT `state name`
    FROM (
        SELECT `state name`, SUM(`cotton production`) AS total_cotton
        FROM agriinfo
        GROUP BY `state name`
        ORDER BY total_cotton DESC
        LIMIT 5
    ) AS RankedStates
),
YearlyProduction AS (
    SELECT year, `state name`, SUM(`cotton production`) AS total_production
    FROM agriinfo
    WHERE `state name` IN (SELECT `state name` FROM TopCottonStates)
    GROUP BY year, `state name`
)
SELECT c1.year, c1.`state name`, c1.total_production,
    ((c1.total_production - c2.total_production) / c2.total_production) * 100 AS growth_rate
FROM YearlyProduction c1
LEFT JOIN YearlyProduction c2
ON c1.`state name` = c2.`state name` AND c1.year = c2.year + 1
ORDER BY c1.`state name`, c1.year;

#6.Districts with the Highest Groundnut Production in 2020
SELECT `dist name`,
SUM(`groundnut production`) AS total_production
FROM agriinfo
WHERE year = 2020 GROUP BY `dist name`
ORDER BY total_production DESC;

#7.Annual Average Maize Yield Across All States
SELECT `state name`, AVG(`maize yield`) AS avg_yield
FROM agriinfo
GROUP BY `state name`
ORDER BY avg_yield DESC
LIMIT 0, 1000;

#8.Total Area Cultivated for Oilseeds in Each State
SELECT `state name`, SUM(`oilseeds area`) AS total_area
FROM agriinfo
GROUP BY `state name`
ORDER BY total_area DESC;

#9.Districts with the Highest Rice Yield
SELECT `dist name`,
SUM(`rice production`) / NULLIF(SUM(`rice area`), 0) AS avg_rice_yield
FROM agriinfo
GROUP BY `dist name`
ORDER BY avg_rice_yield DESC;

#10.Compare the Production of Wheat and Rice for the Top 5 States Over 10 Years
WITH TopStates AS (
    SELECT `state name`
    FROM (
        SELECT `state name`, SUM(`wheat production` + `rice production`) AS total_production
        FROM agriinfo
        WHERE year >= (SELECT MAX(year) - 10 FROM agriinfo)
        GROUP BY `state name`
        ORDER BY total_production DESC
        LIMIT 5
    ) AS top_states
),
YearlyProduction AS(
    SELECT year, `state name`,
        SUM(`rice production`) AS rice_production,
        SUM(`wheat production`) AS wheat_production
    FROM agriinfo
    WHERE `state name` IN (SELECT `state name` FROM TopStates)
    AND year >= (SELECT MAX(year) - 10 FROM agriinfo)
    GROUP BY year, `state name`
    )
SELECT *
from YearlyProduction
order by year, `state name`;